# compas_fea2 simple script approach

This example covers the analysis of a simple 2D frame structure generated completely from scratch. 

In [1]:
from compas_fea2.backends.abaqus.model import Model
from compas_fea2.backends.abaqus.model import Part
from compas_fea2.backends.abaqus.model import Node
from compas_fea2.backends.abaqus.model import ElasticIsotropic
from compas_fea2.backends.abaqus.model import BoxSection
from compas_fea2.backends.abaqus.model import BeamElement
from compas_fea2.backends.abaqus.model import Set

from compas_fea2.backends.abaqus.problem import Problem
from compas_fea2.backends.abaqus.problem import FixedDisplacement
from compas_fea2.backends.abaqus.problem import RollerDisplacementXZ
from compas_fea2.backends.abaqus.problem import PointLoad
from compas_fea2.backends.abaqus.problem import FieldOutput
from compas_fea2.backends.abaqus.problem import GeneralStaticStep

## Model generation

The fist step (of an FEA) is the generation of the analysis `model`. 
A model is always composed by:

1) geometry: nodes, connectivity among nodes, parts in which nodes and elements are groups, etc.
The geometry can either be created from scratch (as in this example) or be imported from other datastructures (networks, meshes, etc).

2) properties:  materials, sections, elements, interactions, etc. Properties are applied to the geometry to simulate the mechanical behaviour. 

In a compas workflow, geometry and properties are provided by the compas_package that creates the structure to analyse. 


In [2]:
model = Model(name='structural_model')

print(model)


compas_fea2 Model object
------------------------
description     : None
dtype           : compas_fea2.backends/Model
elements        : {}
instances       : {}
interactions    : []
materials       : {}
name            : structural_model
nodes           : {}
parts           : {}
releases        : []
sections        : {}
sets            : {}
surfaces        : []



### Parts

`Parts` are subregions of the model that can be considered to be independent from each other. However, similar parts of a model can still be modelled as different `Parts`. 

A good example of a `Part` could be a brick. 

In [3]:
model.add_part(Part(name='part-1'))

print(model.parts['part-1'])


compas_fea2 Part object
-----------------------
dtype           : compas_fea2.backends/Part
elements        : []
elsets          : []
materials       : {}
name            : part-1
nodes           : []
nsets           : []
releases        : []
sections        : {}



Nodes can be added to the `Part` or directly to the `Model` by specifying the name of the `Part` to which they belong.
In this example, we want to create a simple rectangular frame with a node on each side of the frame every 100mm.

In [4]:
for x in range(0, 1100, 100):
    model.add_node(Node(xyz=[x, 0.0, 0.0]), part='part-1')
for y in range(100, 600, 100):
    model.add_node(Node(xyz=[x, y, 0.0]), part='part-1')
for x in range(900, -100, -100):
    model.add_node(Node(xyz=[x, y, 0.0]), part='part-1')
for y in range(400, 0, -100):
    model.add_node(Node(xyz=[x, y, 0.0]), part='part-1')

print(model.parts['part-1'].nodes[5])


compas_fea2 Node object
-----------------------
dtype           : compas_fea2.backends/Node
ex              : None
ey              : None
ez              : None
gkey            : 500.000,0.000,0.000
key             : 5
mass            : None
name            : n-5
x               : 500.0
xyz             : [500.0, 0.0, 0.0]
y               : 0.0
z               : 0.0



We can now add the material properties we plan to use in our analysis. These material will then be available in the 'Model' and can be used in the definition of the structural sections.

In [5]:
# Define materials
model.add_material(ElasticIsotropic(name='mat_A', E=29000, v=0.17, p=2.5e-9))
model.add_material(ElasticIsotropic(name='mat_B', E=25000, v=0.17, p=2.5e-9))

print(model.materials['mat_A'])


compas_fea2 ElasticIsotropic object
-----------------------------------
E               : 29000
G               : 12393.162393162394
dtype           : compas_fea2.backends/ElasticIsotropic
jobdata         : *Material, name=mat_A
*Density
2.5e-09,
*Elastic
29000, 0.17

name            : mat_A
p               : 2.5e-09
unilateral      : None
v               : 0.17



`compas_fea2` uses the same convention as Abaqus where the materials are assigned to the section and not to the elements (sections are assigned to the elements).
In the definition of a structural section you can specify a particular material by passing the material name. The structural properties are computed automatically accordingly to the type of section chosen (`BoxSection` in this case)

In [6]:
# Define sections
model.add_section(BoxSection(name='section_A', material='mat_A', a=20, b=80, t=[5, 5, 2, 3]))
model.add_section(BoxSection(name='section_B', material='mat_B', a=20, b=80, t=2))

print(model.sections['section_A'])


compas_fea2 Section object
--------------------------
dtype           : compas_fea2.backends/BoxSection
guid            : b6e1a0dd-a643-4d4e-8038-f2ea000d7413
material        : mat_A
name            : section_A
properties      : [20, 80, 5, 5, 2, 3]



Structural elements are defined by their type, connectivity and associated section. In this case we use a `BeamElement`, so we need to specify the 2 nodes it is connected to, with the 'BoxSection' defined before.
Elements can be added to the `model` one by one or as a list. 

In [7]:
# Generate elements between nodes
elements = []
for e in range(len(model.parts['part-1'].nodes)-1):
    elements.append((BeamElement(connectivity=[e, e+1], section='section_A')))
model.add_elements(elements=elements, part='part-1')
model.add_element(element=BeamElement(connectivity=[29, 0], section='section_B'), part='part-1')

print(model.parts['part-1'].elements[15])



compas_fea2 BeamElement object
------------------------------
axes            : None
connectivity    : [15, 16]
connectivity_key : 15_16
dtype           : compas_fea2.backends/BeamElement
elset           : None
eltype          : B31
key             : 15
name            : None
orientation     : [0.0, 0.0, -1.0]
section         : section_A
thermal         : None



A `Set` is a unique collection of either nodes or elements and it is used to assign loads or boundary conditions.
In this case we defined two sets for the boundary conditions using node `0` and `10` and one for a point load at node `20`. 

In [8]:
# Define sets for boundary conditions and loads
model.add_instance_set(Set(name='fixed', selection=[0], stype='nset'), instance='part-1-1')
model.add_instance_set(Set(name='roller', selection=[10], stype='nset'), instance='part-1-1')
model.add_instance_set(Set(name='pload', selection=[20], stype='nset'), instance='part-1-1')

In [9]:
print(model)


compas_fea2 Model object
------------------------
description     : None
dtype           : compas_fea2.backends/Model
elements        : {'part-1': [BeamElement(0), BeamElement(1), BeamElement(2), BeamElement(3), BeamElement(4), BeamElement(5), BeamElement(6), BeamElement(7), BeamElement(8), BeamElement(9), BeamElement(10), BeamElement(11), BeamElement(12), BeamElement(13), BeamElement(14), BeamElement(15), BeamElement(16), BeamElement(17), BeamElement(18), BeamElement(19), BeamElement(20), BeamElement(21), BeamElement(22), BeamElement(23), BeamElement(24), BeamElement(25), BeamElement(26), BeamElement(27), BeamElement(28), BeamElement(29)]}
instances       : {'part-1-1': Instance(part-1)}
interactions    : []
materials       : {'mat_A': ElasticIsotropic(mat_A), 'mat_B': ElasticIsotropic(mat_B)}
name            : structural_model
nodes           : {'part-1': [Node(0 - n-0), Node(1 - n-1), Node(2 - n-2), Node(3 - n-3), Node(4 - n-4), Node(5 - n-5), Node(6 - n-6), Node(7 - n-7), Node(8 -

## Problem definition
The second part of the FEA is to define the `Problem` to be solved. To define a `Problem` you need a `Model`, boundary conditions (`bcs`), `laods`, field and history `outputs` and analysis 'steps'.  

In [10]:

# Create the Problem object
problem = Problem(name='test_structure', model=model)

# Assign boundary conditions to the node stes
problem.add_bcs(bcs=[RollerDisplacementXZ(name='bc_roller', bset='roller'),
                        FixedDisplacement(name='bc_fix', bset='fixed')])

# Assign a point load to the node set
problem.add_load(load=PointLoad(name='pload', lset='pload', y=-1000))

# Define the field outputs required
problem.add_field_output(fout=FieldOutput(name='fout'))

# Define the analysis step
problem.add_step(step=GeneralStaticStep(name='gstep', loads=['pload'], field_outputs=['fout']))

print(problem)



compas_fea2 Problem object
--------------------------
bcs             : {'bc_roller': RollerDisplacementXZ(bc_roller), 'bc_fix': FixedDisplacement(bc_fix)}
dtype           : compas_fea2.backends/Problem
field_outputs   : {'fout': <compas_fea2.backends.abaqus.problem.outputs.FieldOutput object at 0x0000027851EA9490>}
history_outputs : {'standard': <compas_fea2.backends.abaqus.problem.outputs.HistoryOutput object at 0x0000027851EA94C0>}
loads           : {'pload': PointLoad(pload)}
model           : 
compas_fea2 Model object
------------------------
description     : None
dtype           : compas_fea2.backends/Model
elements        : {'part-1': [BeamElement(0), BeamElement(1), BeamElement(2), BeamElement(3), BeamElement(4), BeamElement(5), BeamElement(6), BeamElement(7), BeamElement(8), BeamElement(9), BeamElement(10), BeamElement(11), BeamElement(12), BeamElement(13), BeamElement(14), BeamElement(15), BeamElement(16), BeamElement(17), BeamElement(18), BeamElement(19), BeamElement(20), 

Once the `Probelm` is set, you can solve it running the analysis. You need to specify the location of the analysis files generated by Abaqus.
The output log for now is limited to the one automatically generated by Abaqus, but the plan is to expand it to have more info about the status of the analysis, possible warnings or errors, etc. 

In [11]:
# Solve the problem
# my_structure.write_input_file(path='C:/temp/test_structure')
problem.analyse(path='C:/temp/fea2_workshop')

***** Input File generated in: C:\temp\fea2_workshop\test_structure.inp *****

Analysis initiated from SIMULIA established products
Abaqus JOB test_structure
Abaqus 2021
Begin Analysis Input File Processor
11/12/2021 3:01:41 PM
Run pre.exe
11/12/2021 3:01:44 PM
End Analysis Input File Processor
Begin Abaqus/Standard Analysis
11/12/2021 3:01:44 PM
Run standard.exe
11/12/2021 3:01:48 PM
End Abaqus/Standard Analysis
Begin SIM Wrap-up
11/12/2021 3:01:49 PM
Run SMASimUtility.exe
11/12/2021 3:01:50 PM
End SIM Wrap-up
Abaqus JOB test_structure COMPLETED

Abaqus License Manager checked out the following licenses:
Abaqus/Standard checked out 5 tokens from Flexnet server lic-abaqus.ethz.ch.
<315 out of 320 licenses remain available>.

***** Analysis successful - analysis time : 12.315378427505493 s *****


## Results Visualization

...not there yet... :)